In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [7]:
data = pd.read_csv('/Users/kevins/Documents/CSImpact/data/cleaned_prof_data_v4.csv')

In [8]:
data

,Unnamed: 0,name,affiliation,homepage,scholar_id,citations,h-index
0,0,Jiawei Zhang,Univ. of California - Davis,http://jiaweizhang.net,7AkZSJsAAAAJ,4410.0,30.0
1,1,Jiayu Zhou,Michigan State University,http://0xmachine.com,yQKlLTQAAAAJ,5412.0,33.0
2,2,Xinhui Han,Peking University,http://162.105.163.117/index.php?s=/Home/Artic...,NOSCHOLARPAGE,NaN,NaN
3,3,Zhihai Rong,UESTC,http://222.197.183.99/TutorDetails.aspx?id=4088,peq4DgIAAAAJ,2663.0,26.0
4,4,Myung-Soo Kim,Seoul National University,http://3map.snu.ac.kr/mskim,F7VO16YAAAAJ,5205.0,36.0
...,...,...,...,...,...,...,...
20644,20644,Fei Gao,ShanghaiTech University,www.hislab.cn,aDTizY8AAAAJ,1971.0,27.0
20645,20645,Amaldev Manuel,IIT Goa,www.iitgoa.ac.in/~amal,Lb4wzNYAAAAJ,156.0,8.0
20646,20646,Clint P. George,IIT Goa,www.iitgoa.ac.in/~clint,qL6xTH8AAAAJ,114.0,6.0
20647,20647,A. V. Sreejith,IIT Goa,www.iitgoa.ac.in/~sreejithav,6LVNqAMAAAAJ,44.0,5.0


In [9]:
data = data.sort_values('name')

In [11]:
data = data.reset_index(drop = True)

In [15]:
data= data.drop(['Unnamed: 0'], axis = 1)

In [16]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,A Min Tjoa,TU Wien,http://www.ifs.tuwien.ac.at/tjoa,x8qCMhcAAAAJ,6069.0,40.0
1,A. Akbari Azirani,IUST,http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...,pCil4_cAAAAJ,1326.0,18.0
2,A. Aldo Faisal,Imperial College London,https://www.imperial.ac.uk/people/a.faisal,WjHjbrwAAAAJ,6780.0,31.0
3,A. Antony Franklin,IIT Hyderabad,http://www.iith.ac.in/~antony/index.html,LVfqLuoAAAAJ,NaN,NaN
4,A. B. M. Alim Al Islam,BUET,https://sites.google.com/site/abmalimalislam,K-AIPzQAAAAJ,562.0,11.0
...,...,...,...,...,...,...
20644,Ítalo Cunha,UFMG,https://www.dcc.ufmg.br/~cunha,bTAky1EAAAAJ,2036.0,23.0
20645,Óscar Cánovas,University of Murcia,http://webs.um.es/ocanovas/miwiki/doku.php,iAPkCb8AAAAJ,625.0,13.0
20646,Özgür Erkent,Hacettepe University,https://web.cs.hacettepe.edu.tr/~ozgurerkent,5QMAbisAAAAJ,348.0,11.0
20647,Özgür Ulusoy,Bilkent University,http://www.cs.bilkent.edu.tr/~oulusoy,M_vU9pQAAAAJ,4716.0,37.0


In [253]:
earliest_paper_years = []
#total_rows = data.shape[0]
test_data = data.head(100)
total_test_rows = test_data.shape[0]

for i in range(total_test_rows):
    s_id = test_data.iloc[i,3]
    if not pd.isna(test_data.iloc[i,4]):
        html_text = requests.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate').text
        soup = BeautifulSoup(html_text, 'lxml')
        values = soup.find_all('button', id = 'gsc_bpf_more')

        driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
        text1 = driver.page_source
        while True:
            try:
                button = driver.find_element_by_id('gsc_bpf_more')
                time.sleep(0.75)
                button.click()
            except ElementNotInteractableException:
                final_text = driver.page_source
                break
            except NoSuchElementException:
                driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
                break
            text2=driver.page_source
            if text1 == text2:
                final_text = text2
                break
            else:
                text1 = text2

        final_soup = BeautifulSoup(final_text, 'lxml')
        years = final_soup.find_all('td', class_ = 'gsc_a_y')
        years_text = []
        for year in years:
            years_text.append(year.text)

        clean_years = [year_text for year_text in years_text if year_text != '']

        min_y = int(clean_years[0])
        for clean_y in clean_years:
            if int(clean_y) < min_y:
                min_y = int(clean_y)
        earliest_paper_years.append(str(min_y))

    else:
        earliest_paper_years.append('')

/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_89736/2416858570.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_id('gsc_bpf_more')


KeyboardInterrupt: 

In [250]:
from selenium import webdriver
import webbrowser
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time

In [248]:
driver = webdriver.Safari()

In [116]:
#
#
#
#TEST#####
#
#

#1. get html text, make soup
#2. find button
#3. check if button is disabled
#4a. Yes: find earliest paper
#4b No: Click button, go to #3

html_text = requests.get('https://scholar.google.com/citations?hl=en&user=C1skWKgAAAAJ&view_op=list_works&sortby=pubdate').text
soup = BeautifulSoup(html_text, 'lxml')
values = soup.find_all('button', id = 'gsc_bpf_more')

driver.get('https://scholar.google.com/citations?hl=en&user=C1skWKgAAAAJ&view_op=list_works&sortby=pubdate')
text1 = driver.page_source
while True:
    button = driver.find_element_by_id('gsc_bpf_more')
    button.click()
    text2=driver.page_source
    if text1 == text2:
        final_text = text2
        break
    else:
        text1 = text2
        
final_soup = BeautifulSoup(final_text, 'lxml')
years = final_soup.find_all('td', class_ = 'gsc_a_y')
years_text = []
for year in years:
    years_text.append(year.text)

clean_years = [year_text for year_text in years_text if year_text != '']

min_y = int(clean_years[0])
for clean_y in clean_years:
    if int(clean_y) < min_y:
        min_y = int(clean_y)



/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_89736/623857051.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_id('gsc_bpf_more')
